In [1]:
import tensorflow as tf
from tensorflow import keras

import IPython

In [3]:
!pip install -q -U keras-tuner
import kerastuner as kt

     |████████████████████████████████| 71kB 7.2MB/s 


In [4]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [5]:
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

# 모델 정의하기

In [6]:
def model_builder(hp):
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(28, 28)))

    # Tune the number of units in the first Dense layer
    # Choose an optimal value between 32-512
    hp_units = hp.Int('units', min_value = 32, max_value = 512, step = 32)
    
    model.add(keras.layers.Dense(units = hp_units, activation = 'relu'))
    model.add(keras.layers.Dense(10))

    # Tune the learning rate for the optimizer 
    # Choose an optimal value from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4]) 

    model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate),
                    loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True), 
                    metrics = ['accuracy'])

    return model

In [8]:
tuner = kt.Hyperband(model_builder,
                     objective = 'val_accuracy', 
                     max_epochs = 10,
                     factor = 3,
                     directory = 'my_dir',
                     project_name = 'intro_to_kt')

In [9]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)

In [10]:
tuner.search(img_train, label_train, epochs = 10, validation_data = (img_test, label_test), callbacks = [ClearTrainingOutput()])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Trial 30 Complete [00h 00m 41s]
val_accuracy: 0.8711000084877014

Best val_accuracy So Far: 0.8858000040054321
Total elapsed time: 00h 09m 23s
INFO:tensorflow:Oracle triggered exit

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 256 and the optimal learning rate for the optimizer
is 0.001.



In [11]:
# Build the model with the optimal hyperparameters and train it on the data
model = tuner.hypermodel.build(best_hps)
model.fit(img_train, label_train, epochs = 10, validation_data = (img_test, label_test))

Epoch 1/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.4837 - accuracy: 0.8278 - val_loss: 0.4226 - val_accuracy: 0.8537
Epoch 2/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3620 - accuracy: 0.8685 - val_loss: 0.3671 - val_accuracy: 0.8666
Epoch 3/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3249 - accuracy: 0.8805 - val_loss: 0.3806 - val_accuracy: 0.8626
Epoch 4/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3017 - accuracy: 0.8894 - val_loss: 0.3501 - val_accuracy: 0.8725
Epoch 5/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2841 - accuracy: 0.8949 - val_loss: 0.3411 - val_accuracy: 0.8763
Epoch 6/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2686 - accuracy: 0.9010 - val_loss: 0.3346 - val_accuracy: 0.8792
Epoch 7/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2578 - accuracy: 0.9035 - val_loss: 0.3749 - val_accuracy: